<a href="https://colab.research.google.com/github/silverstar0727/pragmatism-AI/blob/main/MLP_HPO(cifar10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q wandb

     |████████████████████████████████| 1.8 MB 4.0 MB/s 
     |████████████████████████████████| 138 kB 17.4 MB/s 
     |████████████████████████████████| 170 kB 17.4 MB/s 
     |████████████████████████████████| 97 kB 5.8 MB/s 
     |████████████████████████████████| 133 kB 17.2 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 62 kB 906 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import argparse
from tensorflow import keras

from wandb.keras import WandbCallback
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=3)

print("x shape")
print(f"train: {x_train.shape}, val: {x_val.shape}, test: {x_test.shape}")
print("y shape")
print(f"train: {y_train.shape}, val: {y_val.shape}, test: {y_test.shape}")

170500096/170498071 [==============================] - 2s 0us/step
x shape
train: (40000, 32, 32, 3), val: (10000, 32, 32, 3), test: (10000, 32, 32, 3)
y shape
train: (40000, 1), val: (10000, 1), test: (10000, 1)


In [14]:
class MLP(tf.keras.Model):
    def __init__(self, act, hid_dim, dropout_rate, n_layer):
        super(MLP, self).__init__()
        self.act = act
        self.hid_dim = hid_dim
        self.dropout_rate = dropout_rate
        self.n_layer = n_layer

        self.flatten = tf.keras.layers.Flatten(input_shape = x_train.shape[1:])
        self.fc = tf.keras.layers.Dense(self.hid_dim, activation=self.act)        
        self.dropout = tf.keras.layers.Dropout(self.dropout_rate)
        self.outputs = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.fc(x)
        x = self.dropout(x)
        x = self.outputs(x)
        return x

In [15]:
def experiment():
    config_defaults = {
        'hid_dim': 512,
        'act': 'relu',
        'lr': 0.001,
        'epochs': 2,
        'batch_size': 256
    }

    wandb.init(project='MLP-HPO', config=config_defaults, magic=True)
    args = wandb.config
    
    # Prepare dataloader
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    trainloader = trainloader.shuffle(1024).batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    valloader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    valloader = valloader.batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    testloader = testloader.batch(args.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    tf.keras.backend.clear_session()
    
    net = MLP(args.act, args.hid_dim, args.dropout_rate, args.n_layer)

    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    optimizer = tf.keras.optimizers.SGD(learning_rate=args.lr)
    metrics = tf.keras.metrics.SparseCategoricalAccuracy()
    net.compile(optimizer, loss, metrics)

    result = net.fit(
        trainloader, 
        epochs=args.epochs, 
        validation_data=valloader, 
        batch_size=128,
        callbacks=[WandbCallback(save_model=False)])

    test_result = net.evaluate(testloader, callbacks=[WandbCallback(save_model=False)])

In [16]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'dropout_rate':{
            'values': [0.2]
        },
        'hid_dim': {
            'values': [512]
        },
        'act': {
            'values': ['relu']
        },
        'lr': {
            'values': [0.1]
        },
        'epochs': {
            'values': [10]
        },
        'batch_size': {
            'values': [256]
        }
    }
}

import wandb
sweep_id = wandb.sweep(sweep_config)

wandb.agent(sweep_id, function=experiment)

Create sweep with ID: 5una3ja4
Sweep URL: https://wandb.ai/jeongmin/uncategorized/sweeps/5una3ja4


wandb: Agent Starting Run: g30lhvrf with config:
wandb: 	act: relu
wandb: 	batch_size: 256
wandb: 	dropout_rate: 0.2
wandb: 	epochs: 10
wandb: 	hid_dim: 512
wandb: 	lr: 0.1
wandb: 	n_layer: 10
wandb: 	out_dim: 10
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/10
157/157 [==============================] - 2s 11ms/step - loss: 2.0810 - sparse_categorical_accuracy: 0.2507 - val_loss: 1.9715 - val_sparse_categorical_accuracy: 0.2781
Epoch 2/10
157/157 [==============================] - 1s 9ms/step - loss: 1.8754 - sparse_categorical_accuracy: 0.3317 - val_loss: 1.8471 - val_sparse_categorical_accuracy: 0.3437
Epoch 3/10
157/157 [==============================] - 1s 9ms/step - loss: 1.7944 - sparse_categorical_accuracy: 0.3652 - val_loss: 1.9328 - val_sparse_categorical_accuracy: 0.2924
Epoch 4/10
157/157 [==============================] - 1s 8ms/step - loss: 1.7436 - sparse_categorical_accuracy: 0.3814 - val_loss: 1.8714 - val_sparse_categorical_accuracy: 0.3245
Epoch 5/10
157/157 [==============================] - 1s 9ms/step - loss: 1.7143 - sparse_categorical_accuracy: 0.3914 - val_loss: 1.7515 - val_sparse_categorical_accuracy: 0.3605
Epoch 6/10
157/157 [==============================] - 1s 9ms/step - loss: 1.6736 - sparse_categoric

epoch,9
loss,1.58754
sparse_categorical_accuracy,0.44055
val_loss,1.65023
val_sparse_categorical_accuracy,0.4132
_runtime,27
_timestamp,1626757185
_step,9
best_val_loss,1.65023
best_epoch,9


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▄▃▃▂▂▂▁▁
sparse_categorical_accuracy,▁▄▅▆▆▇▇▇██
val_loss,▆▄▅▄▂█▅▂▃▁
val_sparse_categorical_accuracy,▁▄▂▃▅▃▄▇▅█
_runtime,▁▁▂▃▄▅▅▆▇█
_timestamp,▁▁▂▃▄▅▅▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: uagejsnt with config:
wandb: 	act: relu
wandb: 	batch_size: 256
wandb: 	dropout_rate: 0.2
wandb: 	epochs: 10
wandb: 	hid_dim: 512
wandb: 	lr: 0.1
wandb: 	n_layer: 20
wandb: 	out_dim: 10
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/10
157/157 [==============================] - 2s 12ms/step - loss: 2.0814 - sparse_categorical_accuracy: 0.2548 - val_loss: 1.9405 - val_sparse_categorical_accuracy: 0.2728
Epoch 2/10
157/157 [==============================] - 1s 8ms/step - loss: 1.8779 - sparse_categorical_accuracy: 0.3249 - val_loss: 2.4546 - val_sparse_categorical_accuracy: 0.1858
Epoch 3/10
157/157 [==============================] - 2s 10ms/step - loss: 1.8114 - sparse_categorical_accuracy: 0.3541 - val_loss: 1.8449 - val_sparse_categorical_accuracy: 0.3498
Epoch 4/10
157/157 [==============================] - 1s 8ms/step - loss: 1.7555 - sparse_categorical_accuracy: 0.3780 - val_loss: 1.8517 - val_sparse_categorical_accuracy: 0.3461
Epoch 5/10
157/157 [==============================] - 1s 9ms/step - loss: 1.7184 - sparse_categorical_accuracy: 0.3902 - val_loss: 1.7266 - val_sparse_categorical_accuracy: 0.3825
Epoch 6/10
157/157 [==============================] - 1s 7ms/step - loss: 1.6914 - sparse_categori

epoch,9
loss,1.60202
sparse_categorical_accuracy,0.43765
val_loss,1.82846
val_sparse_categorical_accuracy,0.3709
_runtime,24
_timestamp,1626757215
_step,9
best_val_loss,1.72662
best_epoch,4


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▄▃▃▂▂▂▁▁
sparse_categorical_accuracy,▁▄▅▆▆▇▇▇██
val_loss,▃█▂▂▁▁▂▁▂▂
val_sparse_categorical_accuracy,▄▁▇▇█▇▇███
_runtime,▁▂▃▃▄▅▆▇██
_timestamp,▁▂▃▃▄▅▆▇██
_step,▁▂▃▃▄▅▆▆▇█


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
